In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import chess
from IPython import display
import pickle

from bot.bots import HaiBotLong
from bot.bots import HaiBotLongs
from bot.model import VFA

import torch
import numpy as np

### import model

In [ ]:
def player_play(board): 
    your_move = None
    while your_move is None: 
        try: 
            your_move = board.parse_uci(input("uci-move:"))
        except: 
            your_move = board.parse_uci(input("invalid move, need valid uci-move:"))
    board.push(your_move)
    outcome = board.outcome()
    return outcome

def bot_play_wrapper(bot, e=0.02): 
    def bot_play(board): 
        action_bot = bot.play(board, e=e, training=False)
        move_bot = board.parse_uci(action_bot)
        board.push(move_bot)
        outcome = board.outcome()
        return outcome
    return bot_play

### helper functions

In [ ]:
def get_single_bot(path, color): 
    bot_vfa = VFA()
    bot_vfa.load_state_dict(torch.load(path))
    bot = HaiBotLong(color=color, vfa=bot_vfa)
    return bot

def get_ensemble_bot(path_ls, color): 
    bot_vfa_ls = []
    for path in path_ls: 
        bot_vfa = VFA()
        bot_vfa.load_state_dict(torch.load(path))
        bot_vfa_ls.append(bot_vfa)
    bot = HaiBotLongs(vfa_ls=bot_vfa_ls, color=color,)
    return bot

In [ ]:
def display_game(bot1, bot2, e=0.00001): 
    board = chess.Board()

    p1 = bot_play_wrapper(bot1, e=e)
    p2 = bot_play_wrapper(bot2)

    display.display(board)
    while True: 
        outcome = p1(board)
        display.clear_output(wait=True)
        display.display(board)
        if outcome is not None:
            break

        # time.sleep(0.2)
        outcome = p2(board)
        display.clear_output(wait=True)
        display.display(board)
        if outcome is not None:
            break
    print(outcome)
    
def play_game(bot1, bot2, e=0.00001): 
    board = chess.Board()

    p1 = bot_play_wrapper(bot1, e=e)
    p2 = bot_play_wrapper(bot2)
    while True: 
        outcome = p1(board)
        if outcome is not None:
            break

        outcome = p2(board)
        if outcome is not None:
            break
            
    print(outcome)
    if outcome.winner is not None: 
        return 1 if outcome.winner else 2
    else: 
        return 0

In [ ]:
def evaluate(ls): 
    ls = np.array(ls)
    draws = (ls==0).sum()
    b1 = (ls==1).sum()
    b2 = (ls==2).sum()
    print(f"draws: {draws}")
    print(f"bot1 wins: {b1}")
    print(f"bot2 wins: {b2}")

### bot game

In [ ]:
# initialize bots
bot1_path = "out/models/v8b/m_810_iter_74550.ckpt"
bot1 = get_single_bot(bot1_path, color="W")

bot2_path = "out/models/v8b/m_690_iter_62338.ckpt"
bot2 = get_single_bot(bot2_path, color="B")


bot_path_ls = [
    "out/models/v8/m_410_iter_34868.ckpt",
    "out/models/v8/m_490_iter_42784.ckpt",
    "out/models/v8b/m_680_iter_61158.ckpt",
    "out/models/v8b/m_690_iter_62338.ckpt",
    # "out/models/v8b/m_690_iter_62338.ckpt",
    "out/models/v8b/m_750_iter_68319.ckpt", 
    "out/models/v8b/m_810_iter_74550.ckpt",
    # "out/models/v8b/m_830_iter_76270.ckpt",
    # "out/models/v8b/m_880_iter_81177.ckpt", 
    "out/models/v8b/m_950_iter_88179.ckpt", 
    # "out/models/v8b/m_1010_iter_93755.ckpt", 

]

bot1 = get_ensemble_bot(bot_path_ls, color="W")

In [ ]:
# display_game(bot1, bot2)

In [ ]:
%%time
ls = []

for i in range(20): 
    ls.append(play_game(bot1, bot2))

In [ ]:
evaluate(ls)